1. Download/Load SP500 stocks prices data

In [5]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-03-24'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-03-28 A        37.037914   39.520000   39.750000   39.139999   39.540001   
           AAL      39.202133   40.910000   41.340000   40.330002   41.000000   
           AAPL     24.040293   26.297501   26.547501   26.264999   26.500000   
           ABBV     39.753651   56.110001   56.700001   55.799999   56.349998   
           ABT      35.243393   40.880001   41.000000   40.610001   40.770000   
...                       ...         ...         ...         ...         ...   
2024-03-22 XYL     129.210007  129.210007  130.479996  128.770004  129.929993   
           YUM     136.229996  136.229996  136.720001  135.649994  136.610001   
           ZBH     127.199997  127.199997  127.650002  125.879997  127.269997   
           ZBRA    291.440002  291.440002  293.869995  290.160004  293.869995   
           ZTS     168.899994  168.899994  171.360001  168.160004  168.770004   

Price                  volume  
date       ticker              
2016-03-28 A        1272700.0  
           AAL      6402800.0  
           AAPL    77645600.0  
           ABBV     3803100.0  
           ABT      3328000.0  
...                       ...  
2024-03-22 XYL       732400.0  
           YUM      1296400.0  
           ZBH      1419200.0  
           ZBRA      201800.0  
           ZTS      2840500.0  

[995648 rows x 6 columns]

2. Calculate features and technical indicators for each stock.

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [8]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])



df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-03-28 A        37.037914   39.520000   39.750000   39.139999   39.540001   
           AAL      39.202133   40.910000   41.340000   40.330002   41.000000   
           AAPL     24.040293   26.297501   26.547501   26.264999   26.500000   
           ABBV     39.753651   56.110001   56.700001   55.799999   56.349998   
           ABT      35.243393   40.880001   41.000000   40.610001   40.770000   
...                       ...         ...         ...         ...         ...   
2024-03-22 XYL     129.210007  129.210007  130.479996  128.770004  129.929993   
           YUM     136.229996  136.229996  136.720001  135.649994  136.610001   
           ZBH     127.199997  127.199997  127.650002  125.879997  127.269997   
           ZBRA    291.440002  291.440002  293.869995  290.160004  293.869995   
           ZTS     168.899994  168.899994  171.360001  168.160004  168.770004   

Price                  volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2016-03-28 A        1272700.0         -0.001531        NaN       NaN   
           AAL      6402800.0         -0.000471        NaN       NaN   
           AAPL    77645600.0         -0.003608        NaN       NaN   
           ABBV     3803100.0         -0.046891        NaN       NaN   
           ABT      3328000.0         -0.008151        NaN       NaN   
...                       ...               ...        ...       ...   
2024-03-22 XYL       732400.0          0.000075  66.372960  4.836604   
           YUM      1296400.0          0.000028  52.736484  4.915549   
           ZBH      1419200.0          0.000097  55.330244  4.826682   
           ZBRA      201800.0          0.000054  59.824982  5.616619   
           ZTS      2840500.0          0.000177  33.482478  5.098090   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-03-28 A            NaN       NaN       NaN       NaN      47.138153  
           AAL          NaN       NaN       NaN       NaN     251.003418  
           AAPL         NaN       NaN       NaN       NaN    1866.622954  
           ABBV         NaN       NaN       NaN       NaN     151.187109  
           ABT          NaN       NaN       NaN       NaN     117.290012  
...                     ...       ...       ...       ...            ...  
2024-03-22 XYL     4.855185  4.873767 -0.170416  1.071474      94.633409  
           YUM     4.935263  4.954977 -0.008496  0.328832     176.608566  
           ZBH     4.845375  4.864068 -0.736258  0.191079     180.522236  
           ZBRA    5.654357  5.692095  0.007843  0.700949      58.812592  
           ZTS     5.209244  5.320399  1.034330 -2.790276     479.760433  

[995648 rows x 14 columns]